In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter='\t', quoting=3)

In [3]:
test = pd.read_csv("testData.tsv", header=0, delimiter='\t', quoting=3)

In [4]:
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter='\t', quoting=3)

In [5]:
print("Read %d labeled reveiews, %d unlabeled training reviews, %d test reviews from the dataset"
      %(train["review"].size,unlabeled_train["review"].size, test["review"].size))

Read 25000 labeled reveiews, 50000 unlabeled training reviews, 25000 test reviews from the dataset


In [6]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [7]:
def review_to_wordlist(raw_review, remove_stopwords=False):
    
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    
    review_text = BeautifulSoup(raw_review).get_text()
    
    #  
    # 2. Remove non-letters
    
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    #
    # 3. Convert words to lower case and split them
    
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    #
    # 5. Return a list of words
    
    return(words)

In [8]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [9]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [10]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


C:\Users\Public\Anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\Public\Anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\Public\Anaconda3\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


C:\Users\Public\Anaconda3\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Public\Anaconda3\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Public\Anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\Publi

In [11]:
# Check how many sentences we have in total - should be around 850,000+
print (len(sentences))

print (sentences[0])

print (sentences[1])


795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [12]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

C:\Users\Public\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-01-26 03:47:26,173 : INFO : 'pattern' package not found; tag filters are not available for English
2019-01-26 03:47:26,185 : INFO : collecting all words and their counts
2019-01-26 03:47:26,186 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-01-26 03:47:26,256 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-01-26 03:47:26,323 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2019-01-26 03:47:26,385 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-01-26 03:47:26,451 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-01-26 03:47:26,511 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-01-26 03:47:26,560 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-01-26 03:47:26,635 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-01-26 03:47:26,721 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-01-26 03:47:26,818 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-01-26 03:47:26,923 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-01-26 03:47:27,006 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

2019-01-26 03:47:31,457 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2019-01-26 03:47:31,517 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-01-26 03:47:31,576 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-01-26 03:47:31,639 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-01-26 03:47:31,696 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-01-26 03:47:31,726 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-01-26 03:47:31,727 : INFO : Loading a fresh vocabulary
2019-01-26 03:47:31,809 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019-01-26 03:47:31,810 : INFO : min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2019-01-26 03:4

2019-01-26 03:48:25,026 : INFO : EPOCH 3 - PROGRESS: at 67.46% examples, 656349 words/s, in_qsize 7, out_qsize 0
2019-01-26 03:48:26,036 : INFO : EPOCH 3 - PROGRESS: at 73.34% examples, 662683 words/s, in_qsize 7, out_qsize 0
2019-01-26 03:48:27,048 : INFO : EPOCH 3 - PROGRESS: at 78.10% examples, 658549 words/s, in_qsize 8, out_qsize 0
2019-01-26 03:48:28,067 : INFO : EPOCH 3 - PROGRESS: at 82.15% examples, 648808 words/s, in_qsize 7, out_qsize 0
2019-01-26 03:48:29,081 : INFO : EPOCH 3 - PROGRESS: at 87.20% examples, 648048 words/s, in_qsize 8, out_qsize 1
2019-01-26 03:48:30,086 : INFO : EPOCH 3 - PROGRESS: at 91.26% examples, 640897 words/s, in_qsize 7, out_qsize 0
2019-01-26 03:48:31,101 : INFO : EPOCH 3 - PROGRESS: at 95.17% examples, 632733 words/s, in_qsize 8, out_qsize 0
2019-01-26 03:48:32,102 : INFO : EPOCH 3 - PROGRESS: at 99.05% examples, 626176 words/s, in_qsize 7, out_qsize 0
2019-01-26 03:48:32,272 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-

In [13]:
model.doesnt_match("man woman child kitchen".split())

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'kitchen'

In [14]:
model.doesnt_match("france england germany berlin".split())

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [15]:
 model.doesnt_match("paris berlin london austria".split())

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'paris'

In [16]:
model.most_similar("man")

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6044820547103882),
 ('lady', 0.5981773138046265),
 ('lad', 0.5942602157592773),
 ('soldier', 0.5302401781082153),
 ('monk', 0.5203213095664978),
 ('men', 0.5164391994476318),
 ('guy', 0.5162383317947388),
 ('sailor', 0.5134319067001343),
 ('businessman', 0.5088698863983154),
 ('person', 0.5057510733604431)]

In [17]:
model.most_similar("queen")

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6953500509262085),
 ('bride', 0.6251678466796875),
 ('eva', 0.6208816766738892),
 ('showgirl', 0.6144726276397705),
 ('belle', 0.6098970770835876),
 ('goddess', 0.5992159843444824),
 ('victoria', 0.5820638537406921),
 ('maria', 0.5806633830070496),
 ('maid', 0.5798587799072266),
 ('mistress', 0.5785026550292969)]

In [22]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2019-01-26 04:52:57,491 : INFO : loading Word2Vec object from 300features_40minwords_10context
2019-01-26 04:52:58,148 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2019-01-26 04:52:58,149 : INFO : setting ignored attribute vectors_norm to None
2019-01-26 04:52:58,151 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2019-01-26 04:52:58,152 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2019-01-26 04:52:58,154 : INFO : setting ignored attribute cum_table to None
2019-01-26 04:52:58,155 : INFO : loaded 300features_40minwords_10context


In [23]:
model["flower"]

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-9.80752483e-02, -5.83812036e-02,  3.21814045e-02,  5.69878630e-02,
       -1.40668005e-01,  7.31226131e-02, -2.14635693e-02,  5.41111492e-02,
        1.31456340e-02, -2.66877078e-02, -2.31220084e-03, -2.19934415e-02,
        2.52331570e-02, -1.69570427e-02, -6.83083534e-02,  5.32113090e-02,
       -1.87292974e-02,  2.80712396e-02, -6.51686937e-02, -1.79825816e-02,
        1.25489049e-02, -2.57144030e-03, -4.89612967e-02, -1.12125855e-02,
        5.82161406e-03,  1.92823932e-02,  8.17447156e-02,  1.51830480e-01,
        2.02049762e-02, -3.40473391e-02, -5.92396371e-02, -1.52695430e-02,
        1.39292069e-02,  6.33001849e-02, -3.90877686e-02,  1.52359083e-02,
       -5.25805801e-02,  1.31830052e-01, -7.48797832e-03, -1.51638668e-02,
        3.26171070e-02, -5.97723201e-02, -1.47278886e-02, -1.54408917e-01,
       -6.01278394e-02, -2.08720453e-02,  1.79009177e-02, -3.07649025e-03,
       -2.75397468e-02, -4.14366461e-02,  5.81660494e-02,  1.02550887e-01,
       -8.60913545e-02, -

In [40]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


In [49]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
        if counter%1000 == 0:
            print ("Review %d of %d" % (counter, len(reviews)))
        
        reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
        
        counter = counter + 1
    return reviewFeatureVecs

In [50]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))
    
num_features = 300

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000


C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [52]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [55]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\s

C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anac

C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
C:\Users\Public\Anac

Time taken for K Means clustering:  726.8042805194855 seconds.


C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


In [61]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))
word_centroid_map_list = list(word_centroid_map.values())

In [66]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print ("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(list(word_centroid_map.values()))):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print (words)


Cluster 0
['egos']

Cluster 1
['laura', 'lisa', 'claire', 'emily', 'karen', 'ellen', 'jenny', 'sally', 'maggie', 'holly', 'christine', 'wendy', 'daisy', 'pamela', 'beth', 'sara', 'stephanie', 'edie', 'cindy', 'gina', 'alicia', 'angie', 'carla', 'katie', 'patty', 'sissy', 'elaine', 'harper', 'stacy', 'allison', 'gwen', 'cathy', 'locke', 'agnes', 'erin', 'jessie', 'josie', 'chloe', 'lori', 'erika', 'cheryl', 'mona', 'nell', 'theresa', 'abigail', 'abby', 'jen', 'eliza', 'lilly', 'tiffany', 'lucinda', 'susie', 'becky']

Cluster 2
['doom', 'gloom']

Cluster 3
['air', 'aircraft', 'transport', 'submarine', 'fleet', 'apollo', 'carrier', 'das', 'formation', 'mutiny', 'squadron']

Cluster 4
['followed', 'replaced', 'interrupted']

Cluster 5
['lambs', 'labyrinth', 'creepshow', 'suspiria', 'raider']

Cluster 6
['property', 'loot']

Cluster 7
['political', 'social', 'politics']

Cluster 8
['restrained', 'subdued', 'expressive', 'relaxed', 'refined', 'talkative', 'racy', 'showy', 'franchot']

Clust

In [67]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [68]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [70]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
